In [1]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
    width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   width:max-content;
}
</style>

In [2]:
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import udf, when, rand, struct, col, lit, current_date, current_timestamp, monotonically_increasing_id
from pyspark.sql.types import StringType, IntegerType, StructField, StructType, DoubleType, ArrayType, FloatType

In [3]:
from ophelia.spark.start import Ophelia
from ophelia.spark.read.spark_read import Read
from ophelia.spark.ml.feature_miner import FeatureMiner

In [4]:
ophelia = Ophelia("CRC Model")
sc = ophelia.Spark.build_spark_context()

02:11:08.151 Ophelia [TAPE] +---------------------------------------------------------------------+
02:11:08.151 Ophelia [INFO] | My name is Ophelia Vendata                                          |
02:11:08.151 Ophelia [INFO] | I am an artificial assistant for data mining & ML engine with spark |
02:11:08.151 Ophelia [INFO] | Welcome to Ophelia spark miner engine                               |
02:11:08.151 Ophelia [INFO] | Lib Version Ophelia.0.0.1                                           |
02:11:08.151 Ophelia [WARN] | V for Vendata...                                                    |
02:11:08.151 Ophelia [TAPE] +---------------------------------------------------------------------+
02:11:08.151 Ophelia [WARN] Initializing Spark Session
02:11:18.343 Ophelia [INFO] Spark Version: 3.0.0
02:11:18.343 Ophelia [INFO] This Is: 'CRC Model' App
02:11:18.344 Ophelia [INFO] Spark Context Initialized Success


In [6]:
spark = ophelia.SparkSession
path = "data/raw/csv/bank.csv"

customer_data = spark.readFile(path, 'csv', header=True, infer_schema=True)
customer_data.printSchema()
customer_data.show(5, False)

02:11:50.416 Ophelia [INFO] Read CSV File From Path: data/raw/csv/bank.csv
root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job       |marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit

In [7]:
string_feature = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']
indexed_df = customer_data.toStringIndex(string_feature)
indexed_df.show(5, False)

02:12:09.885 Ophelia [INFO] Creating Multi String Indexers
02:12:12.687 Ophelia [INFO] Build String Indexer
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+-------------+----------+-------------+-----------+--------------+-------------+
|age|job       |marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|job_index|marital_index|education_index|default_index|housing_index|loan_index|contact_index|month_index|poutcome_index|deposit_index|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+---------+-------------+---------------+-------------+-------------+----------+-------------+-----------+--------------+-------------+
|59 |admin.    |married|secondary|no     |2343   |yes    |no  |unknown|5  |may  |1042    |1   

In [ ]:
import prince

In [ ]:
bankd_pd = pd.read_csv(path)

In [ ]:
cols = ['job', 'marital','education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']
bank_select = bankd_pd[cols]
bank_select

In [ ]:
mca = prince.MCA(
    n_components=5,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='auto',
    random_state=42
)

train_mca = mca.fit(bank_select)

In [ ]:
ax = mca.plot_coordinates(
    X=bank_select,
    ax=None,
    figsize=(8, 8),
    show_row_points=False,
    row_points_size=10,
    show_row_labels=False,
    show_column_points=True,
    column_points_size=30,
    show_column_labels=False,
    legend_n_cols=1
)
#ax.get_figure()#.savefig('images/mca_coordinates.svg')

In [ ]:
mca.eigenvalues_

In [ ]:
mca.column_coordinates(bank_select)

In [ ]:
pd_mca = mca.row_coordinates(bank_select)
pd_mca

In [ ]:
indexed_feature = [f'{c}_index' for c in string_feature]
encoded_df = indexed_df.toOHEncoder(string_feature).drop(*indexed_feature)
encoded_df.show(5, False)
encoded_df.printSchema()

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
np.random.seed(0)

# 10 samples with 5 features
train_features = np.random.rand(10,5)

model = PCA(n_components=2).fit(train_features)
X_pc = model.transform(train_features)

# number of components
n_pcs= model.components_.shape[0]

# get the index of the most important feature on EACH component
# LIST COMPREHENSION HERE
most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

initial_feature_names = ['a','b','c','d','e']

# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))
df

In [ ]:
train_features.shape

In [ ]:
dic

In [ ]:
# esto es el "df" con 10 filas y 5 columnas

train_features.shape

In [ ]:
# esto es el "df" reducido a solo 2 componentes, se mantienen las filas

X_pc.shape

In [ ]:
n_pcs

In [ ]:
# esto es 2 componentes como filas y 5 columnas que son las features originales

(model.components_)

In [ ]:
[np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

# Customer Feature Pattern C.F.P. Model

In [ ]:
from ophelia.spark.ml.unsupervised.FeatureExtraction import SingularValueDecomposition

In [ ]:
svd_df = SingularValueDecomposition(k=10).transform(encoded_df.drop(*string_feature))
svd_df.show(5)

In [ ]:
from ophelia.spark.functions.transpose import TransposeDataFrame

In [ ]:
svd_df.toNarrow(['id'], ['vector_class', 'features']).where(col('vector_class') == 'pca_features').show(5, False)

# K-Means Customer Clusterization  K-M.C.C. Model

In [ ]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol='pca_features', k=5, seed=12345)
model_kmeans = kmeans.fit(svd_df)
kmean_df = model_kmeans.transform(svd_df)
kmean_df.groupBy("prediction").count().orderBy(col("prediction")).show()
kmean_df.show(5)

# K-Means Custering for MCA components

In [ ]:
mca_schema = StructType([StructField('C_1', DoubleType(), True),
                         StructField('C_2', DoubleType(), True),
                         StructField('C_3', DoubleType(), True),
                         StructField('C_4', DoubleType(), True),
                         StructField('C_5', DoubleType(), True),])

from pyspark.sql.functions import monotonically_increasing_id
mca_df = spark.createDataFrame(pd_mca, mca_schema).select(monotonically_increasing_id().alias('row_num'), "*")
mca_df.show(5, False)

In [ ]:
mca_vec = mca_df.toVecAssembler(['C_1', 'C_2', 'C_3', 'C_4', 'C_5']).select('*', col('features').alias('pca_features')).drop('features')
mca_vec.show(5, False)

In [ ]:
mca_kmeans = KMeans(featuresCol='pca_features', k=5, seed=12345)
mca_model_kmeans = mca_kmeans.fit(mca_vec)
mca_kmean_df = mca_model_kmeans.transform(mca_vec)
mca_kmean_df.groupBy("prediction").count().orderBy(col("prediction")).show()
mca_kmean_df.show(5, False)

In [ ]:
x_y_scatter(mca_kmean_df, xy_lims=[-2, 2])

In [ ]:
def extract(row):
    row_tuple = (
        row.id,
        row.features, 
        row.scaled_features,
        row.pca_features,
        row.prediction
    )
    return row_tuple + tuple(row.features.toArray().tolist())

fix_cols = [
    'id',
    'features',
    'scaled_features',
    'pca_features',
    'prediction'
]
extract_feature = kmean_df.rdd.map(extract).toDF(fix_cols)
no_name_cols = extract_feature.drop(*fix_cols).columns
named_cols = encoded_df.columns
cluster_feature = extract_feature.select(*fix_cols, *[col(c).alias(a) for c, a in zip(no_name_cols, named_cols)])

In [ ]:
cluster_feature.show(5)

In [ ]:
import matplotlib.pyplot as plt

def plot_k_pca(X, y, xylims):
    """
    a scatter plot of the 2-dimensional space
    """
    plt.style.use('fivethirtyeight')
    plt.rcParams['figure.figsize'] = 15, 7
    plt.rcParams['font.size'] = 12
    markers = 's', 'x', 'o', '.', '+'
    colors = list(plt.rcParams['axes.prop_cycle'])
    target = np.unique(y)
    for idx, (t, m) in enumerate(zip(target, markers)):
        subset = X[y == t]
        plt.scatter(subset[:, 0], subset[:, 1], s = 50,
                    c = colors[idx]['color'], label = t, marker = m)

    plt.xlim(xylims[0], xylims[1])
    plt.xlabel('PC 1')
    plt.ylim(xylims[0], xylims[1])
    plt.ylabel('PC 2')
    plt.legend(loc = 'upper right')
    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np

def x_y_scatter(df, xy_lims=[-10, 10]):
    X_rdd = df.rdd.map(lambda row: row.pca_features)
    X_pca = np.array(X_rdd.collect())
    y_rdd = df.rdd.map(lambda row: row.prediction)
    y = np.array(y_rdd.collect())
    print(f"\nmat 'X' dim:{X_pca.shape} and vector 'y' dim:{y.shape}\n")
    print(f"clustering with K={X_pca.shape[1]}\n")
    return plot_k_pca(X_pca, y, xy_lims)

x_y_scatter(kmean_df)

In [ ]:
import numpy as np

for k in range(5):
    x_y_scatter(kmean_df.where(col('prediction') == k))